In [ ]:
import pandas as pd
import numpy as np

# Constants
days = 252

file_name = 'input_file.xlsx'

xl = pd.ExcelFile(file_name)
sheet = xl.parse('LMM Currency two')

NumDays = 
periods = 
Numsim = 
Whichsim = 
Floor = 
yrdays = 

# Initializing arrays with zeros
Fwd = np.zeros((Numsim, NumDays + 2, periods + 1))
Dev = np.zeros((Numsim, NumDays))
Timestep = np.zeros(periods + 1)
correl = np.zeros(periods + 1)
fwdvol = np.zeros(periods + 1)
Notional = np.zeros(periods + 1)
Orgswrt = np.zeros(periods + 1)

# Initializing other variables
SumBnd = 0
SumFlt = 0
SumBndAvt = 0
SumFltAvt = 0
OrgSumBnd = 0
OrgSumFlt = 0

# Initializing the variables
for k in range(periods):
    Timestep[k] = sheet.iloc[3 + k, 13] / yrdays
    correl[k] = sheet.iloc[2 + k, 17]
    fwdvol[k] = sheet.iloc[2 + k, 16]
    Orgswrt[k] = sheet.iloc[2 + k, 10]

for k in range(periods - 1):
    Notional[k] = sheet.iloc[2 + k, 12]



